## Librerias

In [1]:
import chess.pgn
import os
import pandas as pd
from tqdm import tqdm
import glob
import re

## Descripción y lógica del script

---

### **Input**

**Archivos PGN crudos:**
- `data/raw/lichess_db_standard_rated_2018-01.pgn`
- `data/raw/lichess_db_standard_rated_2018-02.pgn`
- `data/raw/lichess_db_standard_rated_2018-03.pgn`

**Parámetros clave:**
- **games_per_file:** hasta 5,000,000 partidas por archivo (o hasta EOF).
- **Filtro de ritmo:** solo partidas Blitz (`TimeControl` ∈ {`300+0`, `300`, `5+0`, `5`}).
- **Grupos de ELO válidos:** 1200-1400, 1400-1600, 1600-1800, con |ELO_blancas − ELO_negras| < 100.
- **Secuencia de apertura:** primeros 20 halfmoves en SAN (opcional para análisis posterior).

---

### **Output**

- **CSVs por mes y rango de ELO** (uno por grupo y por archivo PGN):
  - Ejemplo:  
    - `data/processed/chess_data_2018-01_1200-1400.csv`  
    - `data/processed/chess_data_2018-01_1400-1600.csv`  
    - `data/processed/chess_data_2018-01_1600-1800.csv`  
    - (y análogos para los otros meses)

- **CSVs consolidados finales por rango** (concatenación de los tres meses):
  - `data/processed/chess_data_1200-1400_full.csv`
  - `data/processed/chess_data_1400-1600_full.csv`
  - `data/processed/chess_data_1600-1800_full.csv`

**Columnas en los CSVs:**
- `game_id`, `white_player`, `black_player`, `white_elo`, `black_elo`, `elo_group`, `opening`, `eco`, `time_control`, `tipo_de_partida`, `opening_sequence`, `file_tag`

---

### **Lógica de proceso**

- **Lectura y filtrado eficientes** (escalable a 5M partidas/archivo):
  - Lectura secuencial en streaming con `chess.pgn.read_game(pgn)`, evitando cargar el PGN completo en memoria.
  - **Early filtering:** si la partida no es Blitz, se descarta inmediatamente.
  - Para partidas Blitz:
    - Se parsean metadatos mínimos (jugadores, ELOs, ECO, Opening, TimeControl).
    - Se asigna `elo_group` si ambos ELOs caen en el mismo rango y la diferencia es < 100.
    - Se extrae `opening_sequence` con hasta 20 halfmoves en SAN.
    - Se agrega un registro (como dict/fila) a una lista en memoria.
  - Al terminar cada archivo:
    - Se crea un único DataFrame con `pd.concat(records)`.
    - Se filtran registros a los grupos de ELO válidos.
    - Se escribe un CSV por cada `elo_group` presente: `chess_data_{YYYY-MM}_{elo_group}.csv`.

- **Consolidación final por grupo de ELO:**
  - Se buscan los CSVs generados por mes para cada grupo.
  - Se concatenan y guardan como `chess_data_{elo_group}_full.csv`.

---

### **Decisiones de performance y memoria**

- Streaming + filtrado temprano para minimizar uso de memoria.
- Acumulación en una lista y un único `concat` por archivo (evita concatenaciones costosas repetidas).
- Particionar por mes+grupo (picos de memoria acotados; escritura incremental a disco).

---

### **Notas de robustez**

- ELOs inválidos se manejan como `None` y no asignan `elo_group`.
- `file_tag` guarda la marca de mes `YYYY-MM` para trazabilidad.
- Manejo de errores por partida con `try/except` para no interrumpir el batch.
- Este flujo primero separa por mes y rango de ELO (3 archivos por mes) y luego concatena por rango para obtener los CSVs finales.


In [ ]:
class Match:
    def __init__(self, game):
        self.game = game
        headers = game.headers
        self.game_id = headers.get('Site', '').split('/')[-1] if 'Site' in headers else None
        self.white_elo = self._parse_elo(headers.get('WhiteElo'))
        self.black_elo = self._parse_elo(headers.get('BlackElo'))
        self.white_player = headers.get('White', 'Unknown')
        self.black_player = headers.get('Black', 'Unknown')
        self.opening = headers.get('Opening', 'Unknown')
        self.eco = headers.get('ECO', 'Unknown')
        self.time_control = headers.get('TimeControl', 'Unknown')
        self.opening_sequence = self.extract_opening_sequence(20)
        self.tipo_de_partida = self.assign_tipo_de_partida()

    def _parse_elo(self, elo):
        try:
            return int(elo)
        except (TypeError, ValueError):
            return None

    def assign_tipo_de_partida(self):
        """Solo me interesa la partida blitz, las otras las ignoro"""
        blitz_controls = {'300+0', '300', '5+0', '5'}
        if self.time_control in blitz_controls:
            return 'Blitz'
        else:
            return None

    def get_dataframe(self, file_tag=None):
        """Retorna un DataFrame de una fila con los metadatos de la partida"""
        data = {
            'game_id': self.game_id,
            'white_player': self.white_player,
            'black_player': self.black_player,
            'white_elo': self.white_elo,
            'black_elo': self.black_elo,
            'elo_group': assign_to_elo_group(self.white_elo, self.black_elo),
            'opening': self.opening,
            'eco': self.eco,
            'time_control': self.time_control,
            'tipo_de_partida': self.tipo_de_partida,
            'opening_sequence': self.opening_sequence
        }
        if file_tag is not None:
            data['file_tag'] = file_tag
        return pd.DataFrame([data])

    def extract_opening_sequence(self, num_halfmoves=20):
        board = chess.Board()
        moves = []
        for idx, move in enumerate(self.game.mainline_moves()):
            if idx >= num_halfmoves:
                break
            try:
                moves.append(board.san(move))
            except Exception:
                break
            board.push(move)
        return ' '.join(moves)

# Función auxiliar 
def assign_to_elo_group(white_elo, black_elo):
    if white_elo is None or black_elo is None:
        return None
    diff = abs(white_elo - black_elo)
    # Grupos de ELO: 1200-1400, 1400-1600, 1600-1800 (diferencia < 100)
    if 1200 <= white_elo < 1400 and 1200 <= black_elo < 1400 and diff < 100:
        return '1200-1400'
    if 1400 <= white_elo < 1600 and 1400 <= black_elo < 1600 and diff < 100:
        return '1400-1600'
    if 1600 <= white_elo < 1800 and 1600 <= black_elo < 1800 and diff < 100:
        return '1600-1800'
    return 'mezcla'

def analyze_openings(pgn_files, games_per_file=100, output_dir='../data/processed', base_name=None):
    """
    Procesa archivos PGN, filtra partidas Blitz y guarda CSVs por grupo ELO y archivo.
    El nombre del archivo de salida incluye el base_name (ej: chess_data_{base_name}_{elo_group}.csv)
    """

    grupos_validos = ['1200-1400', '1400-1600', '1600-1800']
    os.makedirs(output_dir, exist_ok=True)

    for pgn_file in pgn_files:
        print(f'Procesando {pgn_file}')
        if base_name is not None:
            file_tag = base_name
        else:
            base = os.path.basename(pgn_file)
            match = re.search(r'(\d{4}-\d{2})', base)
            file_tag = match.group(1) if match else base.replace('.pgn', '')

        records = []
        with open(pgn_file, encoding='utf-8') as pgn:
            for _ in tqdm(range(games_per_file), desc=os.path.basename(pgn_file)):
                game = chess.pgn.read_game(pgn)
                if not game:
                    break
                try:
                    m = Match(game)
                    if m.tipo_de_partida == 'Blitz':
                        df = m.get_dataframe(file_tag=file_tag)
                        records.append(df)
                except Exception as e:
                    print(f'Error procesando partida: {e}')
                    continue

        if not records:
            print(f"No se encontraron partidas válidas en {pgn_file}.")
            continue


        all_df = pd.concat(records, ignore_index=True)
        all_df = all_df[all_df['elo_group'].isin(grupos_validos)]

        for elo_grp, group_df in all_df.groupby('elo_group'):
            fname = f'chess_data_{file_tag}_{elo_grp}.csv'
            path = os.path.join(output_dir, fname)
            group_df.to_csv(path, index=False)
            print(f'Se guardó {path} con {len(group_df)} partidas')

In [ ]:
if __name__ == "__main__":
    pgn_files = [
        "../data/raw/lichess_db_standard_rated_2018-01.pgn",
        "../data/raw/lichess_db_standard_rated_2018-02.pgn",
        "../data/raw/lichess_db_standard_rated_2018-03.pgn"
    ]
    output_dir = "../data/processed"

    # Procesar y guardar los 3 grupos por ELO para cada archivo PGN
    for pgn_file in pgn_files:
        print(f"Procesando archivo: {pgn_file}")
        base_match = re.search(r'(\d{4}-\d{2})', os.path.basename(pgn_file))
        base = base_match.group(1) if base_match else os.path.splitext(os.path.basename(pgn_file))[0]
        analyze_openings([pgn_file], games_per_file=5000000, output_dir=output_dir, base_name=base)

    grupos_validos = ['1200-1400', '1400-1600', '1600-1800']
    archivos_por_grupo = {g: [] for g in grupos_validos}

    # Buscar los archivos generados
    for g in grupos_validos:
        pattern = os.path.join(output_dir, f"chess_data_*_{g}.csv")
        archivos_por_grupo[g] = sorted(glob.glob(pattern))

    # Concatenar y guardar 
    for g, files in archivos_por_grupo.items():
        if not files:
            print(f"No se encontraron archivos para el grupo {g}")
            continue
        dfs = [pd.read_csv(f) for f in files]
        df_concat = pd.concat(dfs, ignore_index=True)
        out_path = os.path.join(output_dir, f"chess_data_{g}_full.csv")
        df_concat.to_csv(out_path, index=False)
        print(f"Concatenado y guardado {out_path} ({len(df_concat)} partidas)")


Procesando archivo: ../data/raw/lichess_db_standard_rated_2018-01.pgn
Procesando ../data/raw/lichess_db_standard_rated_2018-01.pgn


lichess_db_standard_rated_2018-01.pgn: 100%|██████████| 5000000/5000000 [3:35:54<00:00, 385.96it/s]  


Se guardó ../data/processed\chess_data_2018-01_1200-1400.csv con 110753 partidas
Se guardó ../data/processed\chess_data_2018-01_1400-1600.csv con 154930 partidas
Se guardó ../data/processed\chess_data_2018-01_1600-1800.csv con 140706 partidas
Procesando archivo: ../data/raw/lichess_db_standard_rated_2018-02.pgn
Procesando ../data/raw/lichess_db_standard_rated_2018-02.pgn


lichess_db_standard_rated_2018-02.pgn: 100%|██████████| 5000000/5000000 [3:33:51<00:00, 389.65it/s]  


Se guardó ../data/processed\chess_data_2018-02_1200-1400.csv con 117641 partidas
Se guardó ../data/processed\chess_data_2018-02_1400-1600.csv con 157109 partidas
Se guardó ../data/processed\chess_data_2018-02_1600-1800.csv con 135814 partidas
Procesando archivo: ../data/raw/lichess_db_standard_rated_2018-03.pgn
Procesando ../data/raw/lichess_db_standard_rated_2018-03.pgn


lichess_db_standard_rated_2018-03.pgn: 100%|██████████| 5000000/5000000 [3:36:28<00:00, 384.97it/s]  


Se guardó ../data/processed\chess_data_2018-03_1200-1400.csv con 117345 partidas
Se guardó ../data/processed\chess_data_2018-03_1400-1600.csv con 156871 partidas
Se guardó ../data/processed\chess_data_2018-03_1600-1800.csv con 137873 partidas
Concatenado y guardado ../data/processed\chess_data_1200-1400_full.csv (345739 partidas)
Concatenado y guardado ../data/processed\chess_data_1400-1600_full.csv (468910 partidas)
Concatenado y guardado ../data/processed\chess_data_1600-1800_full.csv (414393 partidas)
